In [ ]:
import matplotlib.pyplot
import numpy
import pandas
from mpl_toolkits.mplot3d import Axes3D
import seaborn
import matplotlib
import matplotlib.pyplot as pyplot
%pylab inline

matplotlib.style.use('ggplot')

seaborn.set_style('whitegrid')

database_df = pandas.read_csv("./datasets/homocide_reports_1980-2014.csv.gz")

database_df = (database_df.drop(['Record ID', 'Agency Code', 'Victim Ethnicity', 'Perpetrator Ethnicity',
                                 'Record Source'], axis=1))

Victim_Sex = database_df['Victim Sex'].values
Victim_Age = database_df['Victim Age'].values
# Victim_Age = Victim_Age.astype('int16')
Perpetrator_Sex = database_df['Perpetrator Sex'].values
Perpetrator_Age = database_df['Perpetrator Age'].values
# Perpetrator_Age = Perpetrator_Age.astype('int16')
Relationship = database_df['Relationship'].values
Weapon = database_df['Weapon'].values
Solved = database_df['Crime Solved'].values
V_Race = database_df['Victim Race'].values
P_Race = database_df['Perpetrator Race'].values
Crime_Type = database_df['Crime Type'].values
Agency_Name = database_df['Agency Name'].values
Agency_Type = database_df['Agency Type'].values
P_Count = database_df['Perpetrator Count'].values
City = database_df['City'].values
State = database_df['State'].values
Year = database_df['Year'].values
Month = database_df['Month'].values
Rel_Category = database_df['Relationship'].values
W_Category = database_df['Weapon'].values

homicide = pandas.DataFrame(numpy.column_stack((Victim_Sex, Victim_Age, Perpetrator_Sex, Perpetrator_Age,
                                                Relationship, Weapon, Solved, V_Race, P_Race, Crime_Type, Agency_Name,
                                                Agency_Type,
                                                Rel_Category, P_Count, City, State, Year, Month, W_Category)))
homicide.columns = ['Victim_Sex', 'Victim_Age', 'Perpetrator_Sex', 'Perpetrator_Age', 'Relationship',
                    'Weapon', 'Solved', 'V_Race', 'P_Race', 'Crime_Type', 'Agency_Name', 'Agency_Type',
                    'Rel_Category', 'P_Count', 'City', 'State', 'Year', 'Month', 'W_Category']

homicide.loc[(homicide['Relationship'] == 'Wife') | (homicide['Relationship'] == 'Ex-Wife') |
             (homicide['Relationship'] == 'Girlfriend') |
             (homicide['Relationship'] == 'Common-Law Wife'), 'Rel_Category'] = 'Partner-F'

homicide.loc[(homicide['Relationship'] == 'Husband') | (homicide['Relationship'] == 'Ex-Husband') |
             (homicide['Relationship'] == 'Boyfriend') |
             (homicide['Relationship'] == 'Common-Law Husband'), 'Rel_Category'] = 'Partner-M'

homicide.loc[(homicide['Relationship'] == 'Father') | (homicide['Relationship'] == 'In-Law') |
             (homicide['Relationship'] == 'Mother') | (homicide['Relationship'] == 'Stepfather') |
             (homicide['Relationship'] == 'Stepmother'), 'Rel_Category'] = 'Parent'

homicide.loc[(homicide['Relationship'] == 'Daughter') | (homicide['Relationship'] == 'Son') |
             (homicide['Relationship'] == 'Stepdaughter') |
             (homicide['Relationship'] == 'Stepson'), 'Rel_Category'] = 'Children'

homicide.loc[(homicide['Relationship'] == 'Brother') | (homicide['Relationship'] == 'Sister'),
             'Rel_Category'] = 'Sibling'

homicide.loc[(homicide['Relationship'] == 'Employee') | (homicide['Relationship'] == 'Employer'),
             'Rel_Category'] = 'Work'

homicide.loc[(homicide['Relationship'] == 'Boyfriend/Girlfriend') & (homicide['Victim_Sex'] == 'Female'),
             'Rel_Category'] = 'Partner-F'

homicide.loc[(homicide['Relationship'] == 'Boyfriend/Girlfriend') & ((homicide['Victim_Sex'] == 'Male') |
                                                                     (homicide[
                                                                          'Victim_Sex'] == 'Unknown')), 'Rel_Category'] = 'Partner-M'

FV_MP = homicide[(homicide.Victim_Sex == 'Female') & (homicide.Perpetrator_Sex == 'Male')]
FV_FP = homicide[(homicide.Victim_Sex == 'Female') & (homicide.Perpetrator_Sex == 'Female')]
MV_MP = homicide[(homicide.Victim_Sex == 'Male') & (homicide.Perpetrator_Sex == 'Male')]
MV_FP = homicide[(homicide.Victim_Sex == 'Male') & (homicide.Perpetrator_Sex == 'Female')]
FV_UP = homicide[(homicide.Victim_Sex == 'Female') & (homicide.Perpetrator_Sex == 'Unknown')]
MV_UP = homicide[(homicide.Victim_Sex == 'Male') & (homicide.Perpetrator_Sex == 'Unknown')]
UV_UP = homicide[(homicide.Victim_Sex == 'Unknown') & (homicide.Perpetrator_Sex == 'Unknown')]

database_df.head(2)

In [ ]:
pandas.crosstab(homicide.Victim_Sex, homicide.Perpetrator_Sex)

In [ ]:
pyplot.figure(figsize=(12,12),facecolor='#efefef')
seaborn.set()
# ax.set_ticklabels(['0%', '20%', '75%', '100%'])
ax = seaborn.heatmap(pandas.crosstab(homicide.Weapon, homicide.Perpetrator_Sex)
                 .apply(lambda r: r/r.sum(), axis=1), annot=True, fmt=".0%", linewidths=.5,cmap='Blues')
ax.set_title('Weapon Use vs Gender')
cbar = ax.collections[0].colorbar
cbar.set_ticks([0, .25, .50, .75, 1])
cbar.set_ticklabels(['0%', '25%', '50%',  '75%', '100%'])
ax.plot()